# P2. Лучшая комбинация

Ваша задача - обучить стандартный регрессор на основе решающего дерева решать задачу предсказания стоимости бриллианта по набору его признаков, к которым относятся: * Число карат * Цвет * Чистота * Геометрические размеры и т.п.

Датасет можно загрузить по [ссылке](https://drive.google.com/drive/u/0/folders/1CAjs-yQQfHFrAAsX9dy_hXGIyc7wx0CH).

Датасет необходимо предобработать следующим образом:

$\bullet$
Удалить ненужную колонку, дублирующую индекс;

$\bullet$
Преобразовать все данные к числовому типу (если это необходимо)*;

$\bullet$
Воспользуйтесь One-hot encoder для преобразования категориальных признаков. Категориальными признаками в нашем случае служат признаки "cut", "color" и "clarity";

Перемешайте выборку при помощи функции sklearn.utils.shuffle с random_state=42.

$\bullet$
Выберите лучшую комбинацию гиперпараметров из предложенных:

$\bullet$
Критерий ветвления: squared_error, глубина дерева: 12

$\bullet$
Критерий ветвления: friedman_mse, глубина дерева: 16

$\bullet$
Критерий ветвления: poisson, глубина дерева: 22

$\bullet$
Критерий ветвления: squared_error, глубина дерева: 45

$\bullet$
Критерий ветвления: friedman_mse, глубина дерева: 95

$\bullet$
Критерий ветвления: poisson, глубина дерева: 33


Лучшим будет тот критерий, который покажет наилучшее среднее качество с точки зрения метрики $r^2$ при кросс-валидации с cv=10. Random state и random seed установите равными 42.

## Примечания

*Общий алгоритм для этого следующий: вы можете выделить множество всех уникальных значений для каждой колонки, а затем воспользоваться функцией .replace() объекта pandas.DataFrame для замены строчных значений на числовые.

Более правильный способ сделать это - воспользоваться классом 'sklearn.preprocessing.LabelEncoder', подробности в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html). Мы рекомендуем воспользоваться именно LabelEncoder.

In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# Загрузим данные
diamonds = pd.read_csv('TRAIN.csv')

# Удалим ненужную колонку
diamonds.drop('Unnamed: 0', axis=1, inplace=True)

# Преобразуем категориальные признаки
cat_features = ['cut', 'color', 'clarity']
for feature in cat_features:
    le = LabelEncoder()
    diamonds[feature] = le.fit_transform(diamonds[feature])
    diamonds = pd.concat([diamonds, pd.get_dummies(diamonds[feature], prefix=feature)], axis=1)
    diamonds.drop(feature, axis=1, inplace=True)

# Перемешаем выборку
diamonds = shuffle(diamonds, random_state=42)

# Разделим данные на признаки и целевую переменную
X = diamonds.drop('price', axis=1)
y = diamonds['price']

# Разделим данные на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Зададим список параметров для перебора
param_grid = [{'criterion': ['friedman_mse'], 'max_depth': [16]}, {'criterion':['squared_error'], 'max_depth': [12]}, {'criterion':['poisson'], 'max_depth': [22]}, {'criterion':['squared_error'], 'max_depth': [45]}, {'criterion':['friedman_mse'], 'max_depth': [95]}, {'criterion':['poisson'], 'max_depth': [33]}]

# Создадим экземпляр модели
dt = DecisionTreeRegressor(random_state=42)

# Создадим экземпляр GridSearchCV
grid_search = GridSearchCV(dt, param_grid, cv=10, scoring='r2')

# Обучим модель
grid_search.fit(X_train, y_train)

# Выведем лучшие параметры и метрику на тестовой выборке
print(f'Лучшие параметры: {grid_search.best_params_}')
print(f'Метрика на тестовой выборке: {grid_search.score(X_test, y_test)}')

Лучшие параметры: {'criterion': 'squared_error', 'max_depth': 12}
Метрика на тестовой выборке: 0.9727742829430988
